In [1]:
import pandas as pd

In [2]:
movie = pd.read_csv("/Users/hasangulec/Desktop/genel/data science path/Recommendation Systems/Kurs Materyalleri(Tavsiye Sistemleri)/datasets/movie_lens_dataset/movie.csv")
rating = pd.read_csv('/Users/hasangulec/Desktop/genel/data science path/Recommendation Systems/Kurs Materyalleri(Tavsiye Sistemleri)/datasets/movie_lens_dataset/rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


#### Veri setimizde, kullanıcıların puanladığı filmler yer almaktadır.
### Amacımız, Item Based methodunu kullanarak, filmlerin rate yapısına göre kullanıcılara film önerebilmektir.

In [7]:
df.shape # 20 milyon kullanıcının rate'i mevcut.

(20000797, 6)

In [8]:
df["title"].nunique() # Veri setinde 27k film mevcut.

27262

In [9]:
df["title"].value_counts().head() # En çok rate edilen ilk 10 film

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: count, dtype: int64

### Bu aşamada yapılması gereken, belirli sınırda olan filmleri değerlendirme aşamasına almamalıyız. Bunun sebebi, bu işi yapan bir şirketin az rate alan bir film ile işi olmayacaktır.

#### 10k'dan düşük rate alan filmleri bir değişkene atayalım.

In [10]:
comment_counts = pd.DataFrame(df["title"].value_counts())
rare_movies = comment_counts[comment_counts["count"] <= 10000].index
print(rare_movies)

Index(['Ronin (1998)', 'My Best Friend's Wedding (1997)',
       'Flintstones, The (1994)', 'Copycat (1995)', 'Space Jam (1996)',
       'Harry Potter and the Goblet of Fire (2005)', 'Junior (1994)',
       'Perfect Storm, The (2000)', 'Swingers (1996)',
       'Fatal Attraction (1987)',
       ...
       'Satan's Sword (Daibosatsu tôge) (1960)',
       'Blind Massage (Tui na) (2014)', 'Prêt à tout (2014)',
       'Ditchdigger's Daughters, The (1997)', 'A.K. (1985)',
       'Rapture (Arrebato) (1980)',
       'Education of Mohammad Hussein, The (2013)', 'Satanas (2007)',
       'Psychosis (2010)', 'Innocence (2014)'],
      dtype='object', name='title', length=26800)


#### 10k'dan düşük rate alan filmleri df içerisinden çıkartılacaktır.

In [12]:
common_movies = df[~df["title"].isin(rare_movies)]

In [19]:
print("Orijinal Veri Seti : {}, >10k Veri Seti: {}".format(df.shape, common_movies.shape))
print("Orijinal Film Sayısı : {}, >10k Film Sayısı: {}".format(df["title"].nunique(),
                                                               common_movies["title"].nunique()))

# 10k'dan düşük rate'leri çıkarttığımızdan dolayı, film sayısında büyük bir azalış meydana gelmiştir

Orijinal Veri Seti : (20000797, 6), >10k Veri Seti: (9050403, 6)
Orijinal Film Sayısı : 27262, >10k Film Sayısı: 462


## User Movie df oluşturulması

In [20]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")

#### Veri seti içerisinden bir film seçelim ve o filme uygun öneriler yapılmasını isteyelim.

In [24]:
movie_name = "Matrix, The (1999)"
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
Matrix, The (1999)                                           1.000000
Matrix Reloaded, The (2003)                                  0.516906
Matrix Revolutions, The (2003)                               0.449588
Blade (1998)                                                 0.334493
Terminator 2: Judgment Day (1991)                            0.333882
Minority Report (2002)                                       0.332434
Mission: Impossible (1996)                                   0.320815
Lord of the Rings: The Fellowship of the Ring, The (2001)    0.318726
Lord of the Rings: The Two Towers, The (2002)                0.318086
Lord of the Rings: The Return of the King, The (2003)        0.314241
dtype: float64

#### Rastgele bir film seçmek istiyorsak eğer,

In [25]:
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

title
Shawshank Redemption, The (1994)    1.000000
Green Mile, The (1999)              0.388181
Good Will Hunting (1997)            0.359274
Schindler's List (1993)             0.338183
Forrest Gump (1994)                 0.333162
Rain Man (1988)                     0.331133
American History X (1998)           0.324650
Dead Poets Society (1989)           0.324484
Braveheart (1995)                   0.317795
Few Good Men, A (1992)              0.315037
dtype: float64